# Pydeck Earth Engine Introduction

This is an introduction to using [Pydeck](https://pydeck.gl) and [Deck.gl](https://deck.gl) with [Google Earth Engine](https://earthengine.google.com/) in Jupyter Notebooks.

If you wish to run this locally, you'll need to install some dependencies. Installing into a new Conda environment is recommended. To create and enter the environment, run:
```
conda create -n pydeck-ee -c conda-forge python jupyter notebook pydeck earthengine-api requests -y
source activate pydeck-ee
jupyter nbextension install --sys-prefix --symlink --overwrite --py pydeck
jupyter nbextension enable --sys-prefix --py pydeck
```
then open Jupyter Notebook with `jupyter notebook`.

Now in a Python Jupyter Notebook, let's first import required packages:

In [1]:
from pydeck_earthengine_layers import EarthEngineLayer
import pydeck as pdk
import requests
import ee

## Authentication

Using Earth Engine requires authentication. If you don't have a Google account approved for use with Earth Engine, you'll need to request access. For more information and to sign up, go to https://signup.earthengine.google.com/.

If you haven't used Earth Engine in Python before, you'll need to run the following authentication command. If you've previously authenticated in Python or the command line, you can skip the next line.

Note that this creates a prompt which waits for user input. If you don't see a prompt, you may need to authenticate on the command line with `earthengine authenticate` and then return here, skipping the Python authentication.

In [ ]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

## Create Map

Next it's time to create a map. Here we create an `ee.Image` object

In [ ]:
# Initialize objects
ee_layers = []
view_state = pdk.ViewState(latitude=37.7749295, longitude=-122.4194155, zoom=10, bearing=0, pitch=45)

In [4]:
# %%
# Add Earth Engine dataset
# This function masks the input with a threshold on the simple cloud score.


def cloudMask(img):
    cloudscore = ee.Algorithms.Landsat.simpleCloudScore(img).select('cloud')
    return img.updateMask(cloudscore.lt(50))


# Load a Landsat 5 image collection.
collection = ee.ImageCollection('LANDSAT/LT5_L1T_TOA') \
    .filterDate('2008-04-01', '2010-04-01') \
    .filterBounds(ee.Geometry.Point(-122.2627, 37.8735)) \
    .map(cloudMask) \
    .select(['B4', 'B3']) \
    .sort('system:time_start', True)  # Sort the collection in chronological order.

print(collection.size().getInfo())

first = collection.first()
propertyNames = first.propertyNames()
print(propertyNames.getInfo())

uid = first.get('system:id')
print(uid.getInfo())




Then just pass these layers to a `pydeck.Deck` instance, and call `.show()` to create a map:

In [8]:
r = pdk.Deck(layers=ee_layers, initial_view_state=view_state)
r.show()

DeckGLWidget(custom_libraries=[{'libraryName': 'EarthEngineLayerLibrary', 'resourceUri': 'https://cdn.jsdelivr…